In [ ]:
import csv
import pandas as pd
from transliterate import translit, get_available_language_codes

In [ ]:
def delete_j(string):
    newstring = ""
    for i in range(0, len(string)):
        if string[i] == 'J':
            newstring += 'Y'
        else:
            newstring += string[i]
    return newstring

In [ ]:
def edit(string):
    splitted = string.split(' ')
    splitted[0] = translit(splitted[0], 'ru', reversed=True).upper()
    name1 = splitted[1]
    name2 = splitted[2]
    splitted[:] = (value for value in splitted if (value != '' and value != name1 and value != name2))
    name1 = translit(name1, 'ru', reversed=True).upper()
    if len(splitted[13]) == 1:
        splitted.insert(13, "N/A")
    flight_sh = splitted[6]
    if flight_sh[-1] == 'O':
        splitted[6] = flight_sh[:-2]
        splitted.insert(7, "NO")
    else:
        splitted[6] = flight_sh[:-3]
        splitted.insert(7, "YES")
    if len(splitted) == 17:
        for i in range(0, 3):
            splitted.append("N/A")
        splitted.insert(1, name1)
        splitted[0] = delete_j(splitted[0])
        splitted[1] = delete_j(splitted[1])
        return ",".join(splitted)
    if len(splitted[17]) != 1:
        if splitted[17] == "Assistance":
            splitted[17] += ' ' + splitted[18]
            del splitted[18]
        elif splitted[17] == "Travel":
            splitted[17] += ' ' + splitted[18] + ' ' + splitted[19]
            del splitted[18:20]
        else: splitted.insert(17, "N/A")
    if len(splitted) == 18:
        for i in range(0, 2):
            splitted.append("N/A")
        splitted.insert(1, name1)
        splitted[0] = delete_j(splitted[0])
        splitted[1] = delete_j(splitted[1])
        return ",".join(splitted)
    if splitted[18][2] != '#':
        splitted.insert(18, "N/A")
    else:
        splitted[18] += ' ' + splitted[19]
        del splitted[19]
    
    if len(splitted) == 19:
        splitted.append("N/A")
    splitted.insert(1, name1)
    
    splitted[0] = delete_j(splitted[0])
    splitted[1] = delete_j(splitted[1])
        
    return ",".join(splitted)

In [ ]:
tsv = open("Sirena-export-fixed.tab", 'r')
text = tsv.read()
lines = text.split('\n')
output = open("Sirena_edited.csv", 'w')
output.write("PaxLastName,PaxFirstName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,Flight,CodeSh,From,Dest,Code,e-Ticket,TravelDoc,Seat,Meal,TrvCls,Fare,Baggage,PaxAdditionalInfo,AgentInfo\n")
current = 1;
total = len(lines)
print(total)
for line in lines:
    if len(edited_line) and current != total:
        edited_line = edit(line)
        edited_line += "\n"
        output.write(edited_line)
    current += 1
tsv.close()
output.close()

In [ ]:
def norm(string):   
    if type(string) != str:
        return string
    string = string.upper()
    place_yi = string.find('YI')
    if place_yi != -1:
        string = string[:place_yi] + 'YY' + string[place_yi+2:]
        
    place_tc = string.find('TC')
    if place_tc != -1:
        string = string[:place_tc] + 'TS' + string[place_tc+2:]
        
    place_io = string.find('IO')
    if place_io != -1:
        string = string[:place_io] + 'YO' + string[place_io+2:]
        
    place_ii = string.find('II')
    if place_ii != -1:
        string = string[:place_ii] + 'IY' + string[place_ii+2:]
    
    place_x = string.find('X')
    if place_x != -1:
        string = string[:place_x] + 'KS' + string[place_x+1:]
    
    place_ia = string.find('IA')
    if place_ia != -1:
        string = string[:place_ia] + 'YA' + string[place_ia+2:]
    
    place_ia = string.find('AI')
    if place_ia != -1:
        string = string[:place_ia] + 'AY' + string[place_ia+2:]
    
    place_iu = string.find('IU')
    if place_iu != -1:
        string = string[:place_iu] + 'YU' + string[place_iu+2:]
        
    place_iu = string.find('UI')
    if place_iu != -1:
        string = string[:place_iu] + 'UY' + string[place_iu+2:]
    
    place_iu = string.find('EI')
    if place_iu != -1:
        string = string[:place_iu] + 'EY' + string[place_iu+2:]

    place_ump = string.find('\'')
    if place_ump != -1:
        string = string[:place_ump] + string[place_ump+1:]

    return string

In [ ]:
#загрузили всё
sirena_df = pd.read_csv("Sirena_edited.csv")
Senya_df = pd.read_csv("PointzAggregator.csv")
Maxon_df = pd.read_csv("FrequentFlyerForum-Profiles.csv")
excel_df = pd.read_csv('excel_df_correct_names.csv')
Boarding_df = pd.read_csv("BoardingData.csv", sep=';')

#пофиксили все имена
sirena_df['PaxLastName'] = sirena_df['PaxLastName'].apply(norm)
sirena_df['PaxFirstName'] = sirena_df['PaxFirstName'].apply(norm)

Senya_df['LastName'] = Senya_df['LastName'].apply(norm)
Senya_df['Name'] = Senya_df['Name'].apply(norm)

excel_df['FirstName'] = excel_df['FirstName'].apply(norm)
excel_df['LastName'] = excel_df['LastName'].apply(norm)

Boarding_df['PassengerFirstName'] = Boarding_df['PassengerFirstName'].apply(norm)
Boarding_df['PassengerLastName'] = Boarding_df['PassengerLastName'].apply(norm)

In [ ]:
Maxon_df['FirstName'] = Maxon_df['FirstName'].apply(norm)
Maxon_df['LastName'] = Maxon_df['LastName'].apply(norm)

In [284]:
#поменяли названия колонок
sirena_df.rename(columns={'PaxLastName': 'LastName', 'PaxFirstName': 'FirstName'}, inplace=True)
Senya_df.rename(columns={'Name': 'FirstName'}, inplace=True)
excel_df.rename(columns={'FlightNumber': 'Flight'}, inplace=True)
Boarding_df.rename(columns={'PassengerLastName': 'LastName', 'PassengerFirstName': 'FirstName', 'FlightNumber': 'Flight'}, inplace=True)


In [290]:
sirena_df.to_csv('maxrudik_file.csv')
Senya_df.to_csv('senya_file.csv')
excel_df.to_csv('ecxel_file.csv')
Boarding_df.to_csv('prev_csv_file.csv', sep=',')
Maxon_df.to_csv('maxon_file.csv')

In [285]:
sirena_df.head(20)

,LastName,FirstName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,Flight,CodeSh,From,...,Code,e-Ticket,TravelDoc,Seat,Meal,TrvCls,Fare,Baggage,PaxAdditionalInfo,AgentInfo
0,OZEROV,ILDAR,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306,NO,SVO,...,ZBQSPY7360415302044672,9375,53270,NaN,NaN,J,JGRPGN0PC,S,FF#SU 38116280,Go2See
1,KOLOSOV,SAMIR,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323,NO,MMK,...,MHPBBX7398421117936516,2244,645520,NaN,KSML,Y,YRSTUQ,9,FF#FB 284903754,NaN
2,IGNATOVA,SNEZHANA,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481,NO,KJA,...,REDLVB5174973140468001,8115,961316,NaN,NaN,Y,YSTNJL,F,NaN,KupiBilet
3,ZHAROV,PLATON,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,...,NSJNGQ5274206497242737,98,6865148,NaN,NaN,J,JFLXLS0PC,#,FF#FB 884556993,Travelgenio
4,NIKOLSKIY,NIKOLAY,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,...,VWNYGF6247422701565929,4396,926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel
5,GLUSHKOV,KONSTANTIN,NaN,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,...,QSDXJY5874178506968181,4788,422492,NaN,NaN,Y,YGRPPN0PC,8,FF#FB 553284496,KupiBilet
6,KAPUSTIN,ARTEM,1982-10-24,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,...,JJADFB7467749130398378,58,142289,NaN,VLML,Y,YFLXNM2PC,F,NaN,KupiBilet
7,ERSHOVA,LYUBOV,NaN,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,...,UMTOZM2183161939566868,776,380126,NaN,NaN,Y,YGRPYX0PC,#,NaN,Aeroflot
8,TITOVA,ZARINA,NaN,2017-07-29,14:15,2017-07-29,15:45,SU1281,NO,KZN,...,KQIPMS5954073786122008,53,7554162,NaN,NaN,Y,YSTNXJ1PC,8,FF#FB 933538031,OZON.travel
9,BARSUKOV,SAMIR,NaN,2017-07-29,14:15,2017-07-29,15:45,SU1281,NO,KZN,...,WGKZTB2264717979478322,78,271703,NaN,NaN,Y,YGRPPP1PC,F,NaN,Aeroflot


In [286]:
Senya_df.head(20)

,UID,FirstName,LastName,CardType,CardNumber,BonusProgramm,Activities,Code,Date,Departure,Arrival,Fare
0,613142142,YAROMIR,ZVEREV,Airlines,FB 171388778,Flying Blue,Airlines,KE827,2017-08-06,rea,SZX,YGRPZT
1,613142142,YAROMIR,ZVEREV,Airlines,FB 171388778,Flying Blue,Airlines,MU9706,2017-10-26,PEK,BSD,YSTNYV
2,103197717,VITALINA,KOROVINA,Airlines,KE 696768759,Korean Air SKYPASS,Airlines,DL5058,2017-09-11,CHA,ATL,YRSTKY
3,103197717,VITALINA,KOROVINA,Airlines,KE 696768759,Korean Air SKYPASS,Airlines,KE1,2017-04-01,rea,HNL,PRSTRU
4,103197717,VITALINA,KOROVINA,Airlines,KE 696768759,Korean Air SKYPASS,Airlines,DL837,2017-09-13,ATL,HNL,YGRPNG
5,103197717,VITALINA,KOROVINA,Airlines,KE 696768759,Korean Air SKYPASS,Airlines,SU1523,2017-02-12,NUX,SVO,YFLXPS
6,138879468,DANIL,VAVILOV,Airlines,DT 980250352,Delta SkyMiles,Airlines,AF4445,2017-11-11,AJA,ORY,YSTNMG
7,138879468,DANIL,VAVILOV,Airlines,DT 980250352,Delta SkyMiles,Airlines,AZ7545,2017-03-07,FCO,PRG,ASTNZL
8,138879468,DANIL,VAVILOV,Airlines,DT 980250352,Delta SkyMiles,Airlines,AF7331,2017-09-17,MLH,ORY,YSTNOE
9,138879468,DANIL,VAVILOV,Airlines,DT 980250352,Delta SkyMiles,Airlines,AF4102,2017-10-07,ORY,AJA,YSTNJD


In [287]:
Maxon_df.head(300)

,FirstName,LastName,NickName,Sex,Status,programm,Number,Date,Codeshare,Flight,ArrivalCity,ArrivalAirport,ArrivalCountry,DepartureCity,DepartureAirport,DepartureCountry
0,NaN,NaN,FrequentFlyer0,Male,Basic,KE,889215424,2017-03-04,False,DL3377,Pittsburgh PA,PIT,United States of,Boston MA,BOS,United States of
1,NaN,NaN,FrequentFlyer0,Male,Elite+,DT,484697244,2017-03-04,False,DL3377,Pittsburgh PA,PIT,United States of,Boston MA,BOS,United States of
2,NaN,NaN,FrequentFlyer0,Male,Basic,KE,889215424,2017-07-21,False,DL3630,Detroit MI,DTW,United States of,Green Bay WI,GRB,United States of
3,NaN,NaN,FrequentFlyer0,Male,Elite+,DT,484697244,2017-07-21,False,DL3630,Detroit MI,DTW,United States of,Green Bay WI,GRB,United States of
4,NaN,NaN,FrequentFlyer0,Male,Basic,KE,889215424,2017-03-22,False,DL5361,New York NY,LGA,United States of,Pittsburgh PA,PIT,United States of
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,FrequentFlyer7,Male,Elite+,KE,885948676,2017-09-04,True,AF7336,Basel/Mulhouse,MLH,France,Paris,ORY,France
296,NaN,NaN,FrequentFlyer7,Male,Basic,DT,980250352,2017-02-27,True,SU4600,Khabarovsk,KHV,Russian Federation,Nogliki,NGK,Russian Federation
297,NaN,NaN,FrequentFlyer7,Male,Basic,SU,510085965,2017-02-27,True,SU4600,Khabarovsk,KHV,Russian Federation,Nogliki,NGK,Russian Federation
298,NaN,NaN,FrequentFlyer7,Male,Elite,FB,650748133,2017-02-27,True,SU4600,Khabarovsk,KHV,Russian Federation,Nogliki,NGK,Russian Federation


In [288]:
excel_df.head(20)

,Unnamed: 0,Gender,Flight,Date,Time,DepartureCountry,DepartureCity,DepartureAirport,ArrivalCountry,ArrivalCity,ArrivalAirport,AirlineName,LoyaltyStatus,Name,FirstName,LastName
0,0,F,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,lidiya zhdanova,LIDIYA,ZHDANOVA
1,1,M,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,sokolov sviatoslav,SVYATOSLAV,SOKOLOV
2,2,M,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,sukhanov sviatoslav,SVYATOSLAV,SUKHANOV
3,3,M,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,denis zelenin,DENIS,ZELENIN
4,4,F,SU5436,2017-01-01,11:25,russia,vladivostok,VVO,french polynesia,seoul,REA,aeroflot,NaN,irina ostrovskaya,IRINA,OSTROVSKAYA
5,5,F,SU5642,2017-01-01,19:25,russia,khabarovsk,KHV,russia,novosibirsk,OVB,aeroflot,NaN,vera isakova,VERA,ISAKOVA
6,6,F,SU5642,2017-01-01,19:25,russia,khabarovsk,KHV,russia,novosibirsk,OVB,aeroflot,NaN,al'bina zvyagintseva,ALBINA,ZVYAGINTSEVA
7,7,F,SU6330,2017-01-01,17:50,russia,kaliningrad,KGD,russia,saint petersburg,LED,aeroflot,NaN,isakova al'bina,ALBINA,ISAKOVA
8,8,F,SU6330,2017-01-01,17:50,russia,kaliningrad,KGD,russia,saint petersburg,LED,aeroflot,NaN,antonina korol'kova,ANTONINA,KOROLKOVA
9,9,F,SU6330,2017-01-01,17:50,russia,kaliningrad,KGD,russia,saint petersburg,LED,aeroflot,NaN,zolotova svetlana,SVETLANA,ZOLOTOVA


In [289]:
Boarding_df.head(20)

,FirstName,PassengerSecondName,LastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,Flight,CodeShare,Destination
0,SAVELIY,VIKTOROVICH,RUSANOV,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow
1,LEV,MARKOVICH,ISAEV,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
2,NIKOLAY,I.,NIKOLSKIY,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd
3,ANATOLIY,PETROVICH,SHILOV,Male,05/24/1997,2595 919752,WQFFUE,Not presented,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
4,MIROSLAVA,VIACHESLAVOVNA,SEMENOVA,Female,01/31/1976,6775 516990,Not presented,Not presented,Registered,2017-03-12,11:45,SU6284,Own,Moscow
5,ARTEM,EDUARDOVICH,KAPUSTIN,Male,10/24/1982,0058 142289,JJADFB,7467749130398378,Transit,2017-03-12,11:45,SU6284,Own,Moscow
6,ZARINA,E.,TITOVA,Female,12/22/2000,6600 251370,Not presented,5954073786122008,None,2017-07-29,14:15,SU1281,Own,Moscow
7,SAMIR,GORDEEVICH,BARSUKOV,Male,02/21/1995,0078 271703,WGKZTB,2264717979478322,Registered,2017-07-29,14:15,SU1281,Own,Moscow
8,RAMIL,E.,ROZHKOV,Male,02/02/1980,0595 142701,POFVPI,Not presented,Registered,2017-04-07,00:15,SU1482,Own,Krasnoyarsk
9,ALBINA,V.,KONSTANTINOVA,Female,07/04/1997,0916 040520,Not presented,1678047813210634,Registered,2017-04-07,00:15,SU1482,Own,Krasnoyarsk
